# Database

Source: [kaggle](https://www.kaggle.com/hugomathien/soccer)

### Country

name | type
--- | ---
id|INTEGER
name|TEXT

### Match

name | type
--- | ---
id|INTEGER
country_id|INTEGER
league_id|INTEGER
season|TEXT
stage|INTEGER
date|TEXT
match_api_id|INTEGER
home_team_api_id|INTEGER
away_team_api_id|INTEGER
home_team_goal|INTEGER
away_team_goal|INTEGER
home_player_X1|INTEGER
 . . . | INTEGER
home_player_X11|INTEGER
away_player_X1|INTEGER
 . . . | INTEGER
away_player_X11|INTEGER
home_player_Y1|INTEGER
 . . . | INTEGER
home_player_Y11|INTEGER
away_player_Y1|INTEGER
 . . . | INTEGER
away_player_Y11|INTEGER
home_player_1|INTEGER
 . . . | INTEGER
home_player_11|INTEGER
away_player_1|INTEGER
 . . . | INTEGER
away_player_11|INTEGER
goal|TEXT
shoton|TEXT
shotoff|TEXT
foulcommit|TEXT
card|TEXT
cross|TEXT
corner|TEXT
possession|TEXT
B365H|NUMERIC
B365D|NUMERIC
B365A|NUMERIC
BWH|NUMERIC
BWD|NUMERIC
BWA|NUMERIC
IWH|NUMERIC
IWD|NUMERIC
IWA|NUMERIC
LBH|NUMERIC
LBD|NUMERIC
LBA|NUMERIC
PSH|NUMERIC
PSD|NUMERIC
PSA|NUMERIC
WHH|NUMERIC
WHD|NUMERIC
WHA|NUMERIC
SJH|NUMERIC
SJD|NUMERIC
SJA|NUMERIC
VCH|NUMERIC
VCD|NUMERIC
VCA|NUMERIC
GBH|NUMERIC
GBD|NUMERIC
GBA|NUMERIC
BSH|NUMERIC
BSD|NUMERIC
BSA|NUMERIC

### Player_Attributes

name | type
--- | ---
id|INTEGER
player_api_id|INTEGER
player_name|TEXT
player_fifa_api_id|INTEGER
birthday|TEXT
height|INTEGER
weight|INTEGER

### Team_Attributes

name | type
--- | ---
id|INTEGER
team_api_id|INTEGER
team_fifa_api_id|INTEGER
team_long_name|TEXT
team_short_name|TEXT

### League

name | type
--- | ---
id|INTEGER
country_id|INTEGER
name|TEXT

### Player

name | type
--- | ---
id|INTEGER
player_api_id|INTEGER
player_name|TEXT
player_fifa_api_id|INTEGER
birthday|TEXT
height|INTEGER
weight|INTEGER

### Team

name | type
--- | ---
id|INTEGER
team_api_id|INTEGER
team_fifa_api_id|INTEGER
team_long_name|TEXT
team_short_name|TEXT



In [1]:
def dict_factory(cursor, row):
    d = {}
    for idx, col in enumerate(cursor.description):
        if col[0] != "date":
            d[col[0]] = row[idx]
        else:
            d[col[0]] = int(row[idx].split("-", 1)[0])
    return d

In [2]:
# load database
import sqlite3
conn = sqlite3.connect('database.sqlite')
conn.row_factory = dict_factory
c = conn.cursor()

In [3]:
query = """SELECT date, 
            home.id as home_id, 
            away.id as away_id, 
            home_team_goal, 
            away_team_goal, 
            home_player_1, home_player_2, home_player_3, home_player_4, home_player_5, home_player_6, home_player_7, home_player_8, home_player_9, home_player_10, home_player_11, 
            away_player_1, away_player_2, away_player_3, away_player_4, away_player_5, away_player_6, away_player_7, away_player_8, away_player_9, away_player_10, away_player_11 
            FROM match, Team home, Team away 
            WHERE match.home_team_api_id = home.team_api_id AND match.away_team_api_id = away.team_api_id and home_player_1;"""
c.execute(query)
match_dict = c.fetchone()
match_dict

{'date': 2009,
 'home_id': 16,
 'away_id': 6,
 'home_team_goal': 1,
 'away_team_goal': 1,
 'home_player_1': 39890,
 'home_player_2': None,
 'home_player_3': 38788,
 'home_player_4': 38312,
 'home_player_5': 26235,
 'home_player_6': None,
 'home_player_7': None,
 'home_player_8': None,
 'home_player_9': 26916,
 'home_player_10': None,
 'home_player_11': 94289,
 'away_player_1': 34480,
 'away_player_2': 38388,
 'away_player_3': 26458,
 'away_player_4': 13423,
 'away_player_5': 38389,
 'away_player_6': 38798,
 'away_player_7': 30949,
 'away_player_8': 38253,
 'away_player_9': 106013,
 'away_player_10': 38383,
 'away_player_11': 46552}

In [4]:
from Player2Vec import *


for match_dict in c.fetchall():
    home_players = [match_dict["home_player_" + str(i)] for i in range(1, 12)]
    away_players = [match_dict["away_player_" + str(i)] for i in range(1, 12)]

    vecs = []
    for player in home_players + away_players:
        if player:
            c.execute('SELECT player_fifa_api_id FROM player WHERE player_api_id=' + str(player))
            player_fifa_api_id = c.fetchone()["player_fifa_api_id"]
            vec = player2vec(player_fifa_api_id)
            if len(vec):
                vecs.append(vec)
    print(torch.cat(vecs, 0))
    break

torch.Size([4, 42])
